# Update PVPC parsing engine

Old version of the XML file parsing was implemented with `BeautifulSoup`, which is far too powerful and complex for the task at hand. 

Also, and because of this dependency, it can be seen as a _web scraper_, which are now discouraged of being included as new integrations in **HomeAssistant Core**.


### Objective

Make a simpler parser for the xml files downloaded from `https://api.esios.ree.es/archives/80/download?date=`, removing `bs4` dependency and using **`xmltodict`**, which is already in use in some HA integrations.


#### Dependencies for this notebook

- `requests`
- `dateutil`
- `xmltodict` (new xml parser)
- `bs4` (for the old parser)

In [21]:
import json
from datetime import date, datetime
from typing import List, Tuple

from dateutil.parser import parse
from pytz import timezone

import requests
import xmltodict

TZ = timezone("Europe/Madrid")
_RESOURCE = "https://api.esios.ree.es/archives/80/download?date={day:%Y-%m-%d}"

# Get a file sample with today prices
xml_data = requests.get(_RESOURCE.format(day=date.today())).text
print(xml_data[:600])

<PVPCDesgloseHorario xmlns="http://sujetos.esios.ree.es/schemas/2014/04/01/PVPCDesgloseHorario-esios-MP/">
<IdentificacionMensaje v="pvpcdesglosehorario_20200222"/>
<VersionMensaje v="1"/>
<TipoMensaje v="Z55"/>
<TipoProceso v="A01"/>
<TipoClasificacion v="A01"/>
<IdentificacionRemitente codificacion="A01" v="10XES-REE------E"/>
<FuncionRemitente v="A04"/>
<IdentificacionDestinatario codificacion="A01" v="10XES-REE------E"/>
<FuncionDestinatario v="A04"/>
<FechaHoraMensaje v="2020-02-21T19:50:38Z"/>                        
<Horizonte v="2020-02-21T23:00Z/2020-02-22T23:00Z"/>
<SeriesTemporales>


In [22]:
def extract_prices_for_tariff(
    xml_data: str, tz: timezone = TZ, tariff: int = 2
) -> Tuple[date, List[float]]:
    """
    PVPC xml data extractor.
    
    Extract hourly prices for the selected tariff from the xml daily file download
    of the official _Spain Electric Network_ (Red Eléctrica Española, REE) 
    for the _Voluntary Price for Small Consumers_ 
    (Precio Voluntario para el Pequeño Consumidor, PVPC).
    """
    data = xmltodict.parse(xml_data)['PVPCDesgloseHorario']
    
    str_horiz = data['Horizonte']['@v']
    day: date = parse(str_horiz.split("/")[0]).astimezone(TZ).date()
    
    tariff_id = f"Z0{tariff}"
    prices = next(
        filter(
            lambda x: (
                x['TerminoCosteHorario']['@v'] == "FEU" 
                and x['TipoPrecio']['@v'] == tariff_id
            ), 
            data['SeriesTemporales'],
        )
    )
    price_values = [
        round(float(pair["Ctd"]["@v"]), 5) for pair in prices['Periodo']["Intervalo"]
    ]
    return day, price_values


day_prices = extract_prices_for_tariff(xml_data, TZ, tariff=2)
day_prices

(datetime.date(2020, 2, 22),
 [0.0569,
  0.05512,
  0.05344,
  0.05377,
  0.05367,
  0.05381,
  0.05179,
  0.05226,
  0.05294,
  0.05215,
  0.05225,
  0.05128,
  0.11663,
  0.11651,
  0.11638,
  0.11598,
  0.11614,
  0.11725,
  0.11918,
  0.12479,
  0.12749,
  0.12609,
  0.0569,
  0.05602])

In [23]:
# OLD version
from bs4 import BeautifulSoup as Soup


def scrap_xml_official_pvpc_daily_prices(
    html_text: str, tz: timezone = TZ, tariff: int = 2
) -> Tuple[date, List[float]]:
    """
    Scrape XML file content to extract hourly prices for the selected tariff

    Using `bs4` with 'html5lib' parser
    """
    ident_tarifa = "Z0{}".format(tariff)
    ident_precio = "FEU"

    soup_pvpc = Soup(html_text, "html5lib")
    str_horiz = soup_pvpc.find_all("horizonte")[0]["v"]
    ts_st = parse(str_horiz.split("/")[0]).astimezone(tz).date()
    for serie in soup_pvpc.find_all("identificacionseriestemporales"):
        columna = serie.find_next("terminocostehorario")["v"]
        if (
            columna == ident_precio
            and serie.tipoprecio["v"] == ident_tarifa
            and len(serie.find_all("tipoprecio")) > 0
        ):
            values = [round(float(v["v"]), 5) for v in serie.find_all("ctd")]
            return ts_st, values
    return ts_st, []


day_prices_old = scrap_xml_official_pvpc_daily_prices(xml_data, TZ, tariff=2)
day_prices_old

(datetime.date(2020, 2, 22),
 [0.0569,
  0.05512,
  0.05344,
  0.05377,
  0.05367,
  0.05381,
  0.05179,
  0.05226,
  0.05294,
  0.05215,
  0.05225,
  0.05128,
  0.11663,
  0.11651,
  0.11638,
  0.11598,
  0.11614,
  0.11725,
  0.11918,
  0.12479,
  0.12749,
  0.12609,
  0.0569,
  0.05602])

In [24]:
# Comparison
assert day_prices == day_prices_old
%timeit extract_prices_for_tariff(xml_data)
%timeit scrap_xml_official_pvpc_daily_prices(xml_data)

6.15 ms ± 23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
68.9 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Using new resource with JSON file :)

In [32]:
_RESOURCE_JSON = "https://api.esios.ree.es/archives/70/download?date={day:%Y-%m-%d}"


def extract_prices_for_tariff_json(
    json_data: str, tz: timezone = TZ, tariff: int = 2
) -> Tuple[date, List[float]]:
    """
    PVPC json data extractor.
    
    Extract hourly prices for the selected tariff from the JSON daily file download
    of the official _Spain Electric Network_ (Red Eléctrica Española, REE) 
    for the _Voluntary Price for Small Consumers_ 
    (Precio Voluntario para el Pequeño Consumidor, PVPC).
    """
    tariff_keys = {1: "GEN", 2: "NOC", 3: "VHC"}
    key = tariff_keys[tariff]
    
    day = datetime.strptime(json_data["PVPC"][0]["Dia"], "%d/%m/%Y").date()
    price_values = [
        round(float(x[key].replace(",", ".")) / 1000.0, 5) 
        for x in json_data["PVPC"]
    ]
    return day, price_values


json_data = requests.get(_RESOURCE_JSON.format(day=date.today())).json()
print(json_data["PVPC"][:1])

day_prices_json = extract_prices_for_tariff_json(json_data)
day_prices_json

[{'Dia': '22/02/2020', 'Hora': '00-01', 'GEN': '106,17', 'NOC': '56,90', 'VHC': '60,87', 'COFGEN': '0,000110046649000000', 'COFNOC': '0,000181360051000000', 'COFVHC': '0,000156612611000000', 'PMHGEN': '52,03', 'PMHNOC': '49,65', 'PMHVHC': '52,33', 'SAHGEN': '2,19', 'SAHNOC': '2,09', 'SAHVHC': '2,20', 'FOMGEN': '0,03', 'FOMNOC': '0,03', 'FOMVHC': '0,03', 'FOSGEN': '0,17', 'FOSNOC': '0,16', 'FOSVHC': '0,17', 'INTGEN': '0,04', 'INTNOC': '0,04', 'INTVHC': '0,04', 'PCAPGEN': '5,76', 'PCAPNOC': '0,95', 'PCAPVHC': '1,36', 'TEUGEN': '44,03', 'TEUNOC': '2,22', 'TEUVHC': '2,88', 'CCVGEN': '1,92', 'CCVNOC': '1,77', 'CCVVHC': '1,87'}]


(datetime.date(2020, 2, 22),
 [0.0569,
  0.05512,
  0.05344,
  0.05377,
  0.05367,
  0.05381,
  0.05179,
  0.05226,
  0.05294,
  0.05215,
  0.05225,
  0.05128,
  0.11663,
  0.11651,
  0.11638,
  0.11598,
  0.11614,
  0.11725,
  0.11918,
  0.12479,
  0.12749,
  0.12609,
  0.0569,
  0.05602])

In [33]:
assert day_prices == day_prices_json
%timeit extract_prices_for_tariff_json(json_data)

17.6 µs ± 43.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Obviously, the _apparent_ gain in speed is not real, as now the JSON is parsed separately. In equal conditions, XML is faster:

In [46]:
%timeit extract_prices_for_tariff(requests.get(_RESOURCE.format(day=date.today())).text)
%timeit extract_prices_for_tariff_json(requests.get(_RESOURCE_JSON.format(day=date.today())).json())

79.2 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
109 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
from time import monotonic

tic = monotonic()
extract_prices_for_tariff_json(requests.get(_RESOURCE_JSON.format(day=date.today())).json())
print(f"JSON took {monotonic() - tic:.3f} sec")

tic = monotonic()
extract_prices_for_tariff(requests.get(_RESOURCE.format(day=date.today())).text)
print(f"XML took {monotonic() - tic:.3f} sec")

JSON took 0.140 sec
XML took 0.067 sec
